In [1]:
import simple_salesforce
import pandas as pd
import requests
import numpy as np
import datetime
import glob
import os
from simple_salesforce import Salesforce

In [2]:

chapter='Seattle'

In [3]:
sf = Salesforce(username=os.environ['SALESFORCE_EMAIL'],
                password=os.environ['SALESFORCE_PASSWORD'],
                security_token=os.environ['SALESFORCE_TOKEN'],
                instance='https://na88.lightning.force.com/',
                version='42.0')

In [32]:
salesforce_cleaning = {
    'Gates Foundation':'Bill & Melinda Gates Foundation',
    'Russell': 'Russell Investments',
    'Symetra': 'Symetra Financial',
    'Oracle': 'Oracle Corporation',
    'Expedia': 'Expedia Inc.',
    'Microsoft': 'Microsoft Corporation',
    'Microsoft Rewards API': 'Microsoft Corporation',
    'Microsoft Rewards / Give with Bing': 'Microsoft Corporation',
    'McKinsey':'McKinsey & Company'
}

In [20]:
report_files = glob.glob('../DonationReports2/*.csv')

In [21]:
report_files[0]

'../DonationReports2/DonationReport_4TEK6CQ1YP_20210816.csv'

In [24]:
import urllib

def encode_dict(d):
    return {k: urllib.parse.quote(v) for k,v in d.items()}

def process_report(report_df):

    for k,row in report_df.iterrows():
        first_name = row['Donor First Name']
        last_name = row['Donor Last Name']
        donation = row['Total Donation to be Acknowledged']
        match = row['Match Amount']
        donation_date=datetime.datetime.strptime(row['Donation Date'], '%Y-%m-%dT%H:%M:%SZ')
        if first_name == "Not shared by donor":
            first_name = "Anonymous"
            last_name = "Gift"
        if (match>0):
            match_type = 'Employee Giving'
            match_name=f"${match} {row.Company} donation match of {first_name} {last_name} via Benevity on {donation_date.strftime('%Y-%m-%d')}"
        if (donation>0) & (match==0):
            match = donation
            donation=0
            match_type = 'Volunteer Grant'
            match_name=f"${match} {row.Company} volunteer hours match of {first_name} {last_name} via Benevity on {donation_date.strftime('%Y-%m-%d')}"

        new_first = first_name.replace("'", "%5C%27")
        new_last = last_name.replace("'", "%5C%27")
        qry_str = f"SELECT Id, Email FROM Contact WHERE FirstName = '{new_first}' AND LastName = '{new_last}'"
        records=sf.query(qry_str)
        if records['totalSize']==0:
            records=sf.query(f"SELECT Id, Email FROM Contact WHERE Email = '{row['Email']}' or npe01__WorkEmail__c = '{row['Email']}'")        
        if records['totalSize']==0:
            account_data= {
                'Name': f"{last_name} ({first_name})" + " Household",
                'npo02__Formal_Greeting__c': "{} {}".format(first_name,last_name),
                'npo02__Informal_Greeting__c': "{}".format(first_name),
                'RecordTypeId':'0121Y000001QTlbQAG'
            }
            if row['Address']!= 'Not shared by donor':
                account_data['BillingStreet']=row['Address']
                account_data['BillingCity']=row['City']
                account_data['BillingState']=row['State/Province']
                account_data['BillingPostalCode']=row['Postal Code']
            
            account_results=sf.Account.create(account_data)
            assert(account_results['success'])
            email = row['Email']
            if email == 'Not shared by donor':
                email=None
            contact_data={
                'FirstName': first_name,
                'LastName': last_name,
                'Email': email,
                'RecordTypeId': "012i0000000plOsAAI",
                'Chapter__c':chapter,
                'AccountId':account_results['id']
                #'AccountRecordType':'0121Y000001QTlbQAG', 
            }
            contact_results=sf.Contact.create(contact_data)
            assert(contact_results['success'])
            contactID=contact_results['id']
            accountId=account_results['id']
            print(f"needed to create account and contact record for {first_name} {last_name}")

        else:
            contactrecords=records['records']

            contactID=contactrecords[0]['Id']
            contact = sf.Contact.get(contactID)
            accountId = contact['AccountId']
            account = sf.Account.get(accountId)

        companyrecords = sf.query(f"SELECT Id FROM Account WHERE Name = '{row.Company}'")

        if companyrecords['totalSize']==0:
            print(f'company not found: {row.Company}')
            print("Need to create Account with this name")
            companyAccountId = None
        else:
            companyAccountId = companyrecords['records'][0]['Id']
            contactRoleData = None
            donation_data = {
                'RecordTypeId': '012i0000000oqevAAA',
                'Amount': donation,
                'AccountId': accountId,
                'StageName': 'Received',
                'Chapter__c': chapter,
                'Probability': 100.0,
                'CloseDate': donation_date.strftime('%Y-%m-%d'),
                'Type': 'Individual',
                'Check__c': row['Transaction ID'],
                'Name': f"{first_name} {last_name} ${donation} donation via Benevity on {donation_date.strftime('%Y-%m-%d')}"
            }
            if match>0:
                match_data = {
                    'RecordTypeId': '012i0000000oqevAAA',
                    'Amount': match,
                    'AccountId': companyAccountId,
                    'StageName': 'Received',
                    'Chapter__c': chapter,
                    'Probability': 100.0,
                    'CloseDate': donation_date.strftime('%Y-%m-%d'),
                    'Type': match_type,
                    'Check__c': row['Transaction ID'],
                    'Name': match_name
                }
                contactRoleData = {
                    "Role": "Soft Credit",
                    "ContactId": contactID,
                    "OpportunityId": "opportunityId",
                    "IsPrimary":False
                }

            already_imported = sf.query(f"SELECT Id FROM Opportunity WHERE Check__c = '{row['Transaction ID']}'")    
            if already_imported['totalSize']==0:
                print('importing row')
                if donation>0:
                    donation_result=sf.Opportunity.create(donation_data)
                    print(donation_result)
                if match>0:
                    print(match_data)
                    match_result=sf.Opportunity.create(match_data)
                    assert(match_result['success'])
                    contactRoleData["OpportunityId"]=match_result['id']
                    contactrole_result=sf.OpportunityContactRole.create(contactRoleData)
                    print(match_result, contactrole_result)
            else:
                print('skipping row already imported')


In [33]:

# a set of company renames that are necessary to map fields between Salesforce and Benevity systems.
for report_file in report_files[5:6]:
    print(report_file)
    report_df = pd.read_csv(report_file, skiprows=11, skipfooter=4,thousands=',')
    report_df['Company']=report_df.Company.replace(salesforce_cleaning)
    process_report(report_df)

../DonationReports2/DonationReport_5580VJ9PH9_20210816.csv


/opt/anaconda3/envs/mindsmatter/lib/python3.7/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


importing row
{'RecordTypeId': '012i0000000oqevAAA', 'Amount': 2.28, 'AccountId': '0011Y00002WTtK0QAL', 'StageName': 'Received', 'Chapter__c': 'Seattle', 'Probability': 100.0, 'CloseDate': '2020-11-02', 'Type': 'Volunteer Grant', 'Check__c': '28XGV8B4TP', 'Name': '$2.28 Microsoft Corporation volunteer hours match of Anonymous Gift via Benevity on 2020-11-02'}
OrderedDict([('id', '0065d00000utQc4AAE'), ('success', True), ('errors', [])]) OrderedDict([('id', '00K5d00000NSPdWEAX'), ('success', True), ('errors', [])])
importing row
{'RecordTypeId': '012i0000000oqevAAA', 'Amount': 1.72, 'AccountId': '0011Y00002WTtK0QAL', 'StageName': 'Received', 'Chapter__c': 'Seattle', 'Probability': 100.0, 'CloseDate': '2020-11-02', 'Type': 'Volunteer Grant', 'Check__c': '28XUDD2629', 'Name': '$1.72 Microsoft Corporation volunteer hours match of Anonymous Gift via Benevity on 2020-11-02'}
OrderedDict([('id', '0065d00000utQc9AAE'), ('success', True), ('errors', [])]) OrderedDict([('id', '00K5d00000NSPaOEAX

KeyboardInterrupt: 

In [30]:
report_df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,Company,Project,Donation Date,Donor First Name,Donor Last Name,Email,Address,City,State/Province,Postal Code,...,Donation Frequency,Currency,Project Remote ID,Source,Reason,Total Donation to be Acknowledged,Match Amount,Cause Support Fee,Merchant Fee,Fee Comment
1,Microsoft Rewards / Give with Bing,Minds Matter of Seattle,2020-11-02T06:15:52Z,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,...,Unspecified,USD,NaN,NaN,NaN,2.28,0.00,0.00,0.00,NaN
2,Microsoft Rewards / Give with Bing,Minds Matter of Seattle,2020-11-02T06:18:32Z,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,...,Unspecified,USD,NaN,NaN,NaN,1.72,0.00,0.00,0.00,NaN
3,Microsoft,Minds Matter of Seattle,2020-11-05T18:11:33Z,Hana,Kim,HANKI@microsoft.com,Not shared by donor,Not shared by donor,Not shared by donor,98122,...,One Time,USD,NaN,Volunteer,Match,50.00,0.00,0.00,0.00,NaN
4,Okta,Minds Matter of Seattle,2020-11-09T19:38:52Z,Molly,He,molly.he@okta.com,Not shared by donor,Not shared by donor,Not shared by donor,79424,...,One Time,USD,NaN,My Rewards,User Donation,10.00,0.00,0.29,0.00,NaN
5,Google,Minds Matter of Seattle,2020-11-16T21:54:33Z,Sirish,Balaga,sirbalaga@google.com,Not shared by donor,Not shared by donor,Not shared by donor,98033,...,One Time,USD,NaN,My Rewards,User Donation,50.00,0.00,0.00,0.00,NaN
6,Microsoft,Minds Matter of Seattle,2020-11-17T23:20:12Z,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,98144,...,Recurring,USD,NaN,Payroll,User Donation,15.00,15.00,0.00,0.00,NaN
7,Microsoft,Minds Matter of Seattle,2020-11-18T01:10:33Z,Haishan,Zhu,HAIZH@microsoft.com,Not shared by donor,Not shared by donor,Not shared by donor,98006,...,One Time,USD,NaN,Payroll,User Donation,50.00,50.00,0.00,0.00,NaN
8,Microsoft,Minds Matter of Seattle,2020-11-18T01:34:51Z,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,98103,...,Recurring,USD,NaN,Payroll,User Donation,41.67,41.67,0.00,0.00,NaN
9,Google,Minds Matter of Seattle,2020-11-20T18:47:27Z,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,Not shared by donor,98033,...,One Time,USD,NaN,My Rewards,User Donation,400.00,0.00,0.00,0.00,NaN


In [156]:
report_df.loc[2]['Donor First Name']=="Not shared by donor"

True

In [159]:
next(i for i,f in enumerate(report_files) if f == report_file)

30